# W_LEDGER_DS ETL Process
### Oracle EBS GL_LEDGERS to EDW W_LEDGER_DS

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE, '1' AS TENANT_ID,
  2624261 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS, 'US' AS LANGUAGE_BASE
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING: Read source tables and apply transformations
CREATE OR REPLACE TEMP VIEW stg_gl_ledgers AS
SELECT gl.NAME AS name, gl.SHORT_NAME AS short_name, gl.DESCRIPTION AS description,
  gl.PERIOD_SET_NAME AS period_set_name, gl.LAST_UPDATE_DATE AS last_update_date,
  gl.CREATION_DATE AS creation_date, gpt.USER_PERIOD_TYPE AS user_period_type,
  CAST(gl.CHART_OF_ACCOUNTS_ID AS STRING) AS chart_of_accounts,
  COALESCE(gl.CURRENCY_CODE, '__UNASSIGNED__') AS currency_code,
  COALESCE(gl.SLA_ACCOUNTING_METHOD_CODE, '__UNASSIGNED__') AS sla_accounting_method_code,
  COALESCE(gl.LEDGER_CATEGORY_CODE, '__UNASSIGNED__') AS ledger_category_code,
  CAST(gl.CREATED_BY AS STRING) AS created_by_id, CAST(gl.LAST_UPDATED_BY AS STRING) AS changed_by_id,
  CAST(gl.LEDGER_ID AS STRING) AS integration_id
FROM workspace.oracle_ebs.GL_LEDGERS gl
INNER JOIN workspace.oracle_ebs.GL_PERIOD_TYPES gpt ON gl.ACCOUNTED_PERIOD_TYPE = gpt.PERIOD_TYPE
LEFT JOIN workspace.oracle_ebs.XLA_ACCTG_METHODS_TL xam
  ON gl.SLA_ACCOUNTING_METHOD_TYPE = xam.ACCOUNTING_METHOD_TYPE_CODE
  AND gl.SLA_ACCOUNTING_METHOD_CODE = xam.ACCOUNTING_METHOD_CODE
  AND xam.LANGUAGE = (SELECT LANGUAGE_BASE FROM etl_params)
CROSS JOIN etl_params p
WHERE COALESCE(gl.COMPLETE_FLAG, 'Y') = 'Y' AND gl.OBJECT_TYPE_CODE = 'L';

In [ ]:
%sql
-- 3) INSERT: Load data into target table W_LEDGER_DS
INSERT INTO workspace.oracle_edw.w_ledger_ds
SELECT s.name, s.short_name, s.description, s.chart_of_accounts, s.currency_code, s.period_set_name,
  s.sla_accounting_method_code, s.user_period_type, s.ledger_category_code,
  s.created_by_id, s.changed_by_id, try_cast(s.creation_date AS TIMESTAMP), try_cast(s.last_update_date AS TIMESTAMP),
  p.DATASOURCE_NUM_ID, s.integration_id, 'N', p.TENANT_ID
FROM stg_gl_ledgers s CROSS JOIN etl_params p;

In [ ]:
%sql
-- 4) UPDATE CONTROL TABLE
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION' AS package_name,
    'W_LEDGER_DS' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date, CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date FROM etl_params p
) src ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.target_table_name = src.target_table_name, tgt.etl_load_date = src.etl_load_date, tgt.committed = src.committed
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
  src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
-- 5) INSERT HISTORY LOG
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION', 'W_LEDGER_DS', p.ETL_USAGE_CODE,
  p.ETL_PROC_WID, p.EXECUTION_ID, p.ETL_PROC_WID, date_add(current_timestamp(), -p.PRUNE_DAYS), NULL,
  current_timestamp(), CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM etl_params p;

In [ ]:
%sql
-- 6) CLEANUP
DROP VIEW IF EXISTS stg_gl_ledgers;
DROP VIEW IF EXISTS etl_params;